<a href="https://colab.research.google.com/github/KimyYoungWook/DataScience/blob/main/practice/SVM(support_vector_machine)%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기본라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

한글깨짐 방지

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()


fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


문제 정의 : SVM(support vector machine) 사용하여 타이타닉 생존자(1) 사망(0) 예측

[step1] 데이터 준비 -seaborn에서 제공하는 titanic 데이터셋 가져오기

In [ ]:
# load_dataset 함수 사용하여 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

[step2] 데이터 탐색 및 데이터 전처리

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [ ]:
# NaN값이 많은 deck(객실 데크 위치) 열을 삭제, embarked와 내용이 겹치는 embark_town(승선 도시) 열을 삭제
rdf = df.drop(['deck','embark_town'], axis=1)

In [ ]:
#age 열에 나이 데이터가 없는 모든 행을 삭제 ->177 개 NaN삭제
rdf = rdf.dropna(subset=['age'], how= 'any', axis=0)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [ ]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [ ]:
rdf['embarked'].fillna(most_freq, inplace =True)

[step3] 본석에 사용할 속성 선택

In [ ]:
# 분석에 활용할 열(속성) 선택( 생존여부, 객실 등급, 성별,나이,
#타이타닉에 탑승한 형제 자매수, 타이타닉에 탑승한 붐모/자녀수 탑승한곳(항구), C=Cherbourg, Q =Queentstown, S =Southampton)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [ ]:
# 원핫인코딩 - 범주형 데이터를 모형이 인식할수 있도록 숫자형으로 변경
# male,female -> [1,0],[0,1]
# embarked C, Q ,S -> [1,0,0],[0,1,0],[0,0,1]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,onehot_sex], axis =1)
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [ ]:
# embarked C, Q ,S -> [1,0,0],[0,1,0],[0,0,1]
onehot_embarked = pd.get_dummies(ndf['embarked'],prefix='town')
ndf = pd.concat([ndf,onehot_embarked],axis=1)

In [ ]:
ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


[step4] 데이터섯 구분 - 훈련용 검증용

In [ ]:
# ndf -> X,y(문제집,정답지)
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['survived']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=7)
print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


[step5] SVM 분류 모형 - sklearn 사용

In [ ]:
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
# 벡터 공간을 맵핑 하는 함수 -> radial basis function (RBF) 커널을 사용
# 선형(linear), 다항식(poly), 가우시안 RBF(rbf) , 시그모이드(sigmoid)
svm_model = svm.SVC(kernel = 'rbf')# 벡터 공간을 맵핑 하는 함수 -> 기본 rbf 사

In [ ]:
# 훈련 데이터를 사용하여 모형 학습
svm_model.fit(X_train,y_train)

# 테스트 데이터를 사용하여 y_hat 예측
y_hat = svm_model.predict(X_test)

[step6] 모델 성능 평가

In [ ]:
from sklearn import metrics
print("테스트 세트 정확도 : {:.2f}%".format(svm_model.score(X_test,y_test)*100))

테스트 세트 정확도 : 61.40%


In [ ]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test,y_hat)
print(svm_report)
# age가 다른 값들에 비해 너무 큰 공간을 가지고 있어서 성능이 좋지 않음
# -> 스캐일링을 통해 축소 시켜야함(정규화)


              precision    recall  f1-score   support

           0       0.61      0.95      0.74       126
           1       0.67      0.13      0.22        89

    accuracy                           0.61       215
   macro avg       0.64      0.54      0.48       215
weighted avg       0.63      0.61      0.53       215



스캐일링한후 재평가

In [ ]:
# 설명 변수(데이터, 문제집) 정규화 -> 스케일링 조절: 0~1사이의 작은 값으로 축소
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=7)
print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


In [ ]:
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
# 벡터 공간을 맵핑 하는 함수 -> radial basis function (RBF) 커널을 사용
# 선형(linear), 다항식(poly), 가우시안 RBF(rbf) , 시그모이드(sigmoid)
svm_model = svm.SVC(kernel = 'rbf')# 벡터 공간을 맵핑 하는 함수 -> 기본 rbf 사

In [ ]:
# 훈련 데이터를 사용하여 모형 학습
svm_model.fit(X_train,y_train)

# 테스트 데이터를 사용하여 y_hat 예측
y_hat = svm_model.predict(X_test)

In [ ]:
from sklearn import metrics
print("테스트 세트 정확도 : {:.2f}%".format(svm_model.score(X_test,y_test)*100))

테스트 세트 정확도 : 80.47%


In [ ]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test,y_hat)
print(svm_report)
# age가 다른 값들에 비해 너무 큰 공간을 가지고 있어서 성능이 좋지 않음
# -> 스캐일링을 통해 축소 시켜야함(정규화)


              precision    recall  f1-score   support

           0       0.78      0.94      0.85       126
           1       0.87      0.62      0.72        89

    accuracy                           0.80       215
   macro avg       0.82      0.78      0.79       215
weighted avg       0.82      0.80      0.80       215

